# C2_W4_Lab_01_Decision_Trees

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import  pandas as pd

|                                                     |   Ear Shape | Face Shape | Whiskers |   Cat  |
|:---------------------------------------------------:|:---------:|:-----------:|:---------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Present  |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |   Pointy   |  Not Round  |  Present  |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Absent   |    1   |
| <img src="images/6.png" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Absent   |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |   Pointy   |  Round      |  Absent   |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |    Floppy  |   Round     |  Absent   |    0   |
| <img src="images/9.png" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |


We will use **one-hot encoding** to encode the categorical features. They will be as follows:

- Ear Shape: Pointy = 1, Floppy = 0
- Face Shape: Round = 1, Not Round = 0
- Whiskers: Present = 1, Absent = 0

Therefore, we have two sets:

- `X_train`: for each example, contains 3 features:
            - Ear Shape (1 if pointy, 0 otherwise)
            - Face Shape (1 if round, 0 otherwise)
            - Whiskers (1 if present, 0 otherwise)
            
- `y_train`: whether the animal is a cat
            - 1 if the animal is a cat
            - 0 otherwise

In [3]:
X_train = np.array([[1, 1, 1],
[0, 0, 1],
 [0, 1, 0],
 [1, 0, 1],
 [1, 1, 1],
 [1, 1, 0],
 [0, 0, 0],
 [1, 1, 0],
 [0, 1, 0],
 [0, 1, 0]])

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [6]:
# calculate entropy
def entropy(p):
    if p==0 or p==1:
        return 0
    else:
        h= -p*np.log2(p)- (1-p)*np.log2(1-p)
    
    return h

In [7]:
print(entropy(0.5))

1.0


In [8]:
# split the observation based on some feature
def split_data(X,index_feature):
    """
    index 0 feature  => ear
    index 1 feature => face
    index 2 feature => whiskers

    if that feature is there , go to left bucket otherwise right bucket
    """
    m,n = X.shape
    left=[]
    right=[]
    for i in range(m):
        if X[i,index_feature]==1:
            left.append(i)
        else:
            right.append(i)

    return left,right


In [11]:
left,right=split_data(X_train,0)

In [20]:
# calculate the weighted entropy
def weighted_entropy(X,y,left_indices,right_indices):
    m,n= X.shape
    w_left= len(left_indices)/m
    w_right= len(right_indices)/m
    p_left= np.sum(y[left_indices])/len(left_indices)
    p_right= np.sum(y[right_indices])/len(right_indices)

    return w_left*entropy(p_left) + w_right*entropy(p_right)

In [21]:
weighted_entropy(X_train,y_train,left,right)

0.7219280948873623

In [22]:
def information_gain(X, y, left_indices, right_indices):
    p_root= np.sum(y)/len(y)
    H_root= entropy(p_root)

    return H_root - weighted_entropy(X,y,left_indices,right_indices)

In [24]:
information_gain(X_train, y_train, left, right)

0.2780719051126377

In [25]:
for i, feature_name in enumerate(['Ear Shape', 'Face Shape', 'Whiskers']):
    left_indices, right_indices = split_data(X_train, i)
    i_gain = information_gain(X_train, y_train, left_indices, right_indices)
    print(f"Feature: {feature_name}, information gain if we split the root node using this feature: {i_gain:.2f}")

Feature: Ear Shape, information gain if we split the root node using this feature: 0.28
Feature: Face Shape, information gain if we split the root node using this feature: 0.03
Feature: Whiskers, information gain if we split the root node using this feature: 0.12


In [26]:
# we should split on the ear basis , highest information gain

The process is **recursive**, which means we must perform these calculations for each node until we meet a stopping criteria:

- If the tree depth after splitting exceeds a threshold
- If the resulting node has only 1 class
- If the information gain of splitting is below a threshold